In [ ]:
    1. Data
        The project should use a real-world dataset and include a reference to thesource in the report - DONE
    2. Importing
        Import data from a flat file (.csv, .xls, xlsx, .txt, etc.) - DONE
        Retrieve data using online SQL, APIs, or web scraping - DONE - kaggle pull and FIFA ranking scrape
    3. Preparation
        Create pandas DataFrame - DONE
        Sorting, indexing, grouping - NO SORT NO INDEX READ INTO THESE
        Drop duplicates, replace missing values - NO DROP DUP NO REPLACE MISS
        Merge DataFrames - DONE
    4. Analysis
        Conditional statements, looping, groupby - NO COND OR LOOP DO THIS IN SQUAD FUNCTION, YES GROUPING
        Define a custom function to create reusable code - TO DO FOR MATCH FUNCTIONS
        Use NumPy functions - NO NUMPY
        Dictionary or Lists - LISTS USED FOR WC TEAMS
    5. Visualisation
        Generate at least two charts using Matplotlib or Seaborn - YES DESCRIBED BELOW, DO SOME ADD ANALYSIS ONES AT BEGINNING IE DIST (WC TEAMS AND NOT) ETC
    6. Insights
        Derive five valuable insights from the analysis - BEST TEAMS ARE HIGH VAL+LUES RATINGS STRENGTH ETC
        Justify your insights with reference to the charts or analysis _ USE GRAPHS
    7. Machine Learning
        Describe what kind of prediction you could perform in future using machine learning and/or deep learning. - INCLUDE MORE DTA IE OLD WORLD CUP EVEN CLUB MATCHES
        Would you use classification or regression methods? - NO TO LOOK INTO
    

In [ ]:
## plan

Take each player:
    Group by Position:
        Weighted forumla that is normalised 70-100 between:
            World Cup Value - player value (average of four years)
            Current transfermarkt summised value
            Current fifa rating
            Potential Rating
        Get each players rating.
    Group by Nationality
        Take the best X players for each position (ie: 3 goalkeeper etc)
        Build a 25 man squad (look for players not included from lower clubs that have been ommited (added these back in seperatley) maybe come back and do fulls scope of players)
    Create a squad rating
        DEF rating
        MID rating
        ATT rating
        FIFA world ranking - web scrape
    Create function that will play each against eachother
        Play group stages loaded in
        Play round of 16
        Play round of 8
        Play semis
        Play final
            Form Increased if you win
                For each match:
                    Team rating will increase/remain/decrease per result (will need recalculated):
                    Win +2 per rating
                    Draw 0 per rating
                    Loss -1 per rating
Visuals of best team on average with:
    Team Averages:
        Strength
        Skills 
        Etc
    Best and Worst Team XI's: 
        (Copy FYP) - use the image links
        Overall
        Potential etc
    Best Players not included in the 25 man squad
    Best Players in last couple World Cups
    Compare my rankins to FIFA ranks
    

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import opendatasets as od

import random
from scipy.stats import norm

### Loading Data in from Kaggle API

f51999f5f297523ab16349d4112bd4ff

In [93]:
od.download("https://www.kaggle.com/datasets/davidcariboo/player-scores")

Skipping, found downloaded files in "./player-scores" (use force=True to force download)


### Reading in the Kaggle tables for Transfer Markt

In [8]:
valuations = (r'/Users/mallon/Desktop/UCD Data Analytics for Finance/player-scores/player_valuations.csv')
players = (r'/Users/mallon/Desktop/UCD Data Analytics for Finance/player-scores/players.csv')
competitions = (r'/Users/mallon/Desktop/UCD Data Analytics for Finance/player-scores/competitions.csv')
clubs = (r'/Users/mallon/Desktop/UCD Data Analytics for Finance/player-scores/clubs.csv')

In [9]:
players= pd.read_csv(players)
valuations= pd.read_csv(valuations)
competitions= pd.read_csv(competitions)
clubs= pd.read_csv(clubs)

In [11]:
print(players["country_of_citizenship"].unique())

['France' 'Iran' 'Germany' 'Russia' 'Spain' 'England' 'Morocco' 'Togo'
 'Ukraine' 'Netherlands' 'Slovakia' 'DR Congo' 'Australia' 'Egypt'
 'Senegal' 'Jamaica' 'Croatia' 'Romania' 'Moldova' 'Ghana' 'Poland'
 'Estonia' 'Slovenia' 'Ireland' 'Serbia' 'Sweden' 'Brazil' 'Norway'
 'Denmark' 'United States' 'Angola' 'Philippines' 'Finland' 'Nigeria'
 'Afghanistan' 'Cameroon' 'Tunisia' 'Hungary' 'North Macedonia' 'Uruguay'
 'Chile' 'Argentina' 'Korea, South' 'Scotland' 'Armenia'
 'Bosnia-Herzegovina' 'Czech Republic' 'Belarus' 'Bulgaria' 'Italy'
 'Portugal' 'Switzerland' 'Turkey' 'Uzbekistan' 'Paraguay' 'Austria'
 'St. Lucia' 'Sierra Leone' 'Albania' "Cote d'Ivoire" 'Colombia' 'Iceland'
 'Chad' 'India' 'Azerbaijan' 'Montenegro' 'Peru' 'Algeria' 'Costa Rica'
 'Belgium' 'Uganda' 'Lithuania' 'Georgia' 'Canada' 'Mali' 'Malaysia'
 'Faroe Islands' 'Greece' 'Cyprus' 'Latvia' 'Liberia' 'Lebanon'
 'Mozambique' 'Venezuela' 'Cape Verde' 'Central African Republic'
 'Zimbabwe' 'Panama' 'Guinea-Bissau' 'Guya

In [12]:
#current players only
still_playing = players['last_season'].isin([2020,2021, 2022])
players = players[still_playing]
players.head()

,player_id,pretty_name,club_id,club_pretty_name,current_club_id,country_of_citizenship,country_of_birth,date_of_birth,position,sub_position,...,height_in_cm,market_value_in_gbp,highest_market_value_in_gbp,agent_name,contract_expiration_date,domestic_competition_id,club_name,image_url,last_season,url
15104,29392,Kjartan Finnbogason,2414,Ac Horsens,2414,Iceland,Iceland,1986-07-09,Attack,Centre-Forward,...,188,180000.0,315000.0,Olafur Gardarsson,2023-10-16,DK1,ac-horsens,NaN,2020,https://www.transfermarkt.co.uk/kjartan-finnbo...
15105,49841,Hamdi Harbaoui,29228,Royal Excel Mouscron,29228,Tunisia,Tunisia,1985-01-05,Attack,Centre-Forward,...,186,180000.0,2250000.0,Creative & Management Group,NaN,BE1,royal-excel-mouscron,NaN,2020,https://www.transfermarkt.co.uk/hamdi-harbaoui...
15106,61637,Emmanuel Riviere,4083,Fc Crotone,4083,Martinique,Martinique,1990-03-03,Attack,Centre-Forward,...,182,405000.0,5400000.0,Cheikh Fall,NaN,IT1,fc-crotone,NaN,2020,https://www.transfermarkt.co.uk/emmanuel-rivie...
15107,194549,Simy,4083,Fc Crotone,4083,Nigeria,Nigeria,1992-05-07,Attack,Centre-Forward,...,198,2700000.0,7200000.0,NaN,2022-06-30,IT1,fc-crotone,NaN,2020,https://www.transfermarkt.co.uk/simy/profil/sp...
15108,707381,Alem Sabic,2414,Ac Horsens,2414,Denmark,Denmark,2003-01-03,Attack,Centre-Forward,...,184,NaN,NaN,football scouting and transfer,NaN,DK1,ac-horsens,NaN,2020,https://www.transfermarkt.co.uk/alem-sabic/pro...


In [14]:
print(players["country_of_citizenship"].unique())

['Iceland' 'Tunisia' 'Martinique' 'Nigeria' 'Denmark' 'Turkey'
 "Cote d'Ivoire" 'Colombia' 'Ukraine' 'Scotland' 'Canada' 'Moldova'
 'DR Congo' 'Kosovo' 'Italy' 'Albania' 'Brazil' 'Greece' 'Croatia' 'Peru'
 'Netherlands' 'United States' 'Belgium' 'Curacao' 'Sierra Leone'
 'Romania' 'Serbia' 'Cameroon' 'Uruguay' 'Finland' 'Germany'
 'Bosnia-Herzegovina' 'Senegal' 'Czech Republic' 'Portugal' 'Bolivia'
 'Austria' 'Poland' 'Hungary' 'Argentina' 'Slovenia' 'France' 'Venezuela'
 'Switzerland' 'Togo' 'Iraq' 'Mali' 'Egypt' 'Morocco' 'Sweden' 'Spain'
 'Ghana' 'Libya' 'Japan' 'Paraguay' 'Cape Verde' 'Algeria' 'Suriname'
 'Ecuador' 'Kenya' 'Benin' 'Lithuania' 'Montenegro' 'Zimbabwe'
 'Northern Ireland' 'England' 'Ireland' 'New Zealand' 'Norway' 'Congo'
 'Guadeloupe' 'Iran' 'Georgia' 'Belarus' 'Aruba' 'Luxembourg' 'Mozambique'
 'The Gambia' 'Faroe Islands' 'Wales' 'Russia' 'Kazakhstan' 'Gabon'
 'Neukaledonien' 'Armenia' 'Chile' 'Rwanda' 'Costa Rica' 'Jamaica'
 'St. Kitts & Nevis' 'Australia' 'Israe

In [15]:
valuations.head()

,date,datetime,dateweek,player_id,current_club_id,market_value,player_club_domestic_competition_id
0,2013-07-22,2013-07-22,2013-07-22,254016,855,90000,FR1
1,2014-01-14,2014-01-14,2014-01-13,254016,855,90000,FR1
2,2008-08-25,2008-08-25,2008-08-25,51053,23,90000,L1
3,2011-05-30,2011-05-30,2011-05-30,51053,23,90000,L1
4,2011-08-26,2011-08-26,2011-08-22,51053,23,225000,L1


In [17]:
joined = pd.merge(left = players, right = valuations, how = 'inner', on = 'player_id' )
#firsttierplayersonly (revise later if no players for country)


In [ ]:
joined = pd.merge(left = joined, right = clubs, how = 'inner', on = 'club_id' )
joined = pd.merge(left = joined, right = competitions, how = 'inner', left_on='player_club_domestic_competition_id', right_on='competition_id' )

In [18]:
print(joined["country_of_citizenship"].unique())

['Iceland' 'Tunisia' 'Martinique' 'Nigeria' 'Turkey' "Cote d'Ivoire"
 'Denmark' 'Colombia' 'Canada' 'Moldova' 'DR Congo' 'Ukraine' 'Scotland'
 'Kosovo' 'Italy' 'Albania' 'Brazil' 'Greece' 'Croatia' 'Peru'
 'Netherlands' 'United States' 'Belgium' 'Curacao' 'Sierra Leone'
 'Romania' 'Serbia' 'Cameroon' 'Uruguay' 'Finland' 'Germany'
 'Bosnia-Herzegovina' 'Senegal' 'Czech Republic' 'Portugal' 'Bolivia'
 'Austria' 'Poland' 'Hungary' 'Argentina' 'Slovenia' 'France' 'Venezuela'
 'Switzerland' 'Togo' 'Iraq' 'Mali' 'Egypt' 'Morocco' 'Sweden' 'Spain'
 'Ghana' 'Libya' 'Japan' 'Paraguay' 'Cape Verde' 'Algeria' 'Suriname'
 'Ecuador' 'Kenya' 'Benin' 'Lithuania' 'Montenegro' 'Zimbabwe'
 'Northern Ireland' 'England' 'Ireland' 'New Zealand' 'Norway' 'Congo'
 'Guadeloupe' 'Iran' 'Georgia' 'Belarus' 'Aruba' 'Luxembourg' 'Mozambique'
 'The Gambia' 'Faroe Islands' 'Wales' 'Russia' 'Kazakhstan' 'Gabon'
 'Neukaledonien' 'Armenia' 'Chile' 'Rwanda' 'Costa Rica' 'Jamaica'
 'St. Kitts & Nevis' 'Australia' 'Israe

In [374]:
#first_tier = joined['sub_type'] == 'first_tier'
toptierplayers = joined

In [375]:
toptierplayers['date'] = pd.to_datetime(toptierplayers['date'], errors='coerce')

In [376]:
toptierplayers['Year']= toptierplayers['date'].dt.year
grouped_multiple = toptierplayers.groupby(['player_id', 'Year']).agg({'market_value': ['mean']})
grouped_multiple.columns = ['market_value_mean']
grouped_multiple['market_value_mean']= grouped_multiple['market_value_mean'].astype(int)
players_avg_value_per_year = grouped_multiple.reset_index()
players_avg_value_per_year.head()

,player_id,Year,market_value_mean
0,2857,2004,100000
1,2857,2005,516666
2,2857,2006,900000
3,2857,2007,1000000
4,2857,2008,1266666


In [377]:
list=[x for x in range(2003, 2022+1)]
WorldCupList = {'World Cup': ['2006','2006','2006','2006', '2010', '2010','2010','2010','2014','2014','2014','2014','2018','2018','2018','2018','2022','2022','2022','2022'], 'Year': list}
WorldCupYears = pd.DataFrame(WorldCupList)  

In [378]:
players_avg_value_per_wc = pd.merge(players_avg_value_per_year, WorldCupYears, left_on='Year', right_on='Year')
players_avg_value_per_wc = players_avg_value_per_wc.groupby(['player_id', 'World Cup']).agg({'market_value_mean': ['mean', 'min', 'max']})
players_avg_value_per_wc.columns = ['market_value_mean', 'market_value_min', 'market_value_max']
players_avg_value_per_wc['market_value_mean']= players_avg_value_per_wc['market_value_mean'].astype(int)
players_avg_value_per_wc = players_avg_value_per_wc.reset_index()

In [379]:
players_avg_value_per_wc = players_avg_value_per_wc.groupby(['player_id', 'World Cup']).agg({'market_value_mean': ['mean', 'min', 'max']})
players_avg_value_per_wc.columns = ['market_value_mean', 'market_value_min', 'market_value_max']
players_avg_value_per_wc = players_avg_value_per_wc.reset_index()

In [380]:
players_avg_value_per_wc

,player_id,World Cup,market_value_mean,market_value_min,market_value_max
0,2857,2006,505555.0,505555,505555
1,2857,2010,1129166.0,1129166,1129166
2,2857,2014,708333.0,708333,708333
3,2857,2018,681250.0,681250,681250
4,2857,2022,477083.0,477083,477083
...,...,...,...,...,...
20578,1048097,2022,50000.0,50000,50000
20579,1048954,2022,50000.0,50000,50000
20580,1050249,2022,75000.0,75000,75000
20581,1050852,2022,50000.0,50000,50000


In [381]:
toptierplayers_per_wc = pd.merge(toptierplayers,players_avg_value_per_wc,on = 'player_id')

In [382]:
toptierplayers_per_wc.columns

Index(['player_id', 'pretty_name_x', 'club_id', 'club_pretty_name',
       'current_club_id_x', 'country_of_citizenship', 'country_of_birth',
       'date_of_birth', 'position', 'sub_position', 'name_x', 'foot',
       'height_in_cm', 'market_value_in_gbp', 'highest_market_value_in_gbp',
       'agent_name', 'contract_expiration_date', 'domestic_competition_id_x',
       'club_name', 'image_url', 'last_season', 'url_x', 'date', 'datetime',
       'dateweek', 'current_club_id_y', 'market_value',
       'player_club_domestic_competition_id', 'name_y', 'pretty_name_y',
       'domestic_competition_id_y', 'total_market_value', 'squad_size',
       'average_age', 'foreigners_number', 'foreigners_percentage',
       'national_team_players', 'stadium_name', 'stadium_seats',
       'net_transfer_record', 'coach_name', 'url_y', 'competition_id',
       'pretty_name', 'type', 'sub_type', 'country_id', 'country_name',
       'country_latitude', 'country_longitude', 'domestic_league_code', 'name',

In [398]:
WC_2022_teams = ['Qatar', 'Germany', 'Denmark', 'Brazil', 'France', 'Belgium', 'Croatia', 'Spain', 'Serbia', 'England', 'Switzerland', 'Netherlands', 'Argentina', 'Iran', 'Korea, South', 'Japan', 'Saudi Arabia', 'Ecuador', 'Uruguay', 'Canada', 'Ghana', 'Senegal', 'Portugal', 'Poland', 'Tunisia', 'Morocco', 'Cameroon', 'United States', 'Mexico', 'Wales', 'Australia', 'Costa Rica']

In [399]:
print(toptierplayers["country_of_citizenship"].unique())

['England' 'Czech Republic' 'Iceland' 'Scotland' 'Wales' 'Netherlands'
 'Ireland' 'Colombia' 'Brazil' 'Nigeria' 'Senegal' 'Germany' 'Sweden'
 'Slovakia' 'Morocco' 'DR Congo' "Cote d'Ivoire" 'Chile'
 'Northern Ireland' 'Belgium' 'Austria' 'Finland' 'United States' 'Poland'
 'Denmark' 'Greece' 'Zambia' 'Romania' 'Paraguay' 'Ecuador' 'Ghana'
 'Spain' 'Japan' 'Argentina' 'Italy' 'Norway' 'Dominican Republic'
 'France' 'Cameroon' 'Grenada' 'Jamaica' 'Albania' 'Iran' 'Serbia' 'Mali'
 'Mexico' 'Portugal' 'Montenegro' 'Korea, South' 'Croatia' 'Venezuela'
 'Switzerland' 'Bosnia-Herzegovina' 'Ukraine' 'Zimbabwe' 'Gabon' 'Algeria'
 'Uruguay' 'Egypt' 'Guinea' 'Tunisia' 'Iraq' 'Estonia' 'New Zealand'
 'Turkey' 'Israel' 'Montserrat' 'Angola' 'Bulgaria' 'Russia' 'Kosovo'
 'North Macedonia' 'Slovenia' 'Martinique' 'Equatorial Guinea'
 'The Gambia' 'Sierra Leone' 'Australia' 'Lithuania' 'Uzbekistan'
 'Suriname' 'Cyprus' 'Moldova' 'Togo' 'Hungary' 'Armenia' 'Georgia'
 'Congo' 'Philippines' 'Costa Rica' 

In [400]:
print(toptierplayers_per_wc["country_of_citizenship"].unique())
print(TopTier_2022_WC_Eligble_Players["country_of_citizenship"].unique())

['England' 'Czech Republic' 'Iceland' 'Scotland' 'Wales' 'Netherlands'
 'Ireland' 'Colombia' 'Brazil' 'Nigeria' 'Senegal' 'Germany' 'Sweden'
 'Slovakia' 'Morocco' 'DR Congo' "Cote d'Ivoire" 'Chile'
 'Northern Ireland' 'Belgium' 'Austria' 'Finland' 'United States' 'Poland'
 'Denmark' 'Greece' 'Zambia' 'Romania' 'Paraguay' 'Ecuador' 'Ghana'
 'Spain' 'Japan' 'Argentina' 'Italy' 'Norway' 'Dominican Republic'
 'France' 'Cameroon' 'Grenada' 'Jamaica' 'Albania' 'Iran' 'Serbia' 'Mali'
 'Mexico' 'Portugal' 'Montenegro' 'Korea, South' 'Croatia' 'Venezuela'
 'Switzerland' 'Bosnia-Herzegovina' 'Ukraine' 'Zimbabwe' 'Gabon' 'Algeria'
 'Uruguay' 'Egypt' 'Guinea' 'Tunisia' 'Iraq' 'Estonia' 'New Zealand'
 'Turkey' 'Israel' 'Montserrat' 'Angola' 'Bulgaria' 'Russia' 'Kosovo'
 'North Macedonia' 'Slovenia' 'Martinique' 'Equatorial Guinea'
 'The Gambia' 'Sierra Leone' 'Australia' 'Lithuania' 'Uzbekistan'
 'Suriname' 'Cyprus' 'Moldova' 'Togo' 'Hungary' 'Armenia' 'Georgia'
 'Congo' 'Philippines' 'Costa Rica' 

In [401]:
bale = toptierplayers_per_wc[toptierplayers_per_wc["pretty_name_x"]== "Neymar"]

In [402]:
TopTier_2022_WC_Eligble_Players = toptierplayers_per_wc[toptierplayers_per_wc.country_of_citizenship.isin(WC_2022_teams)]

In [403]:
TopTier_2022_WC_Eligble_Players =TopTier_2022_WC_Eligble_Players[['World Cup','player_id','pretty_name_x','country_of_citizenship','position','sub_position','date_of_birth','name','market_value_mean','market_value_min','market_value_max']]

In [404]:
TopTier_2022_WC_Eligble_Players.head()

,World Cup,player_id,pretty_name_x,country_of_citizenship,position,sub_position,date_of_birth,name,market_value_mean,market_value_min,market_value_max
0,2010,63200,Ashley Barnes,England,Attack,Centre-Forward,1989-10-30,premier-league,90000.0,90000,90000
1,2014,63200,Ashley Barnes,England,Attack,Centre-Forward,1989-10-30,premier-league,714375.0,714375,714375
2,2018,63200,Ashley Barnes,England,Attack,Centre-Forward,1989-10-30,premier-league,3018750.0,3018750,3018750
3,2022,63200,Ashley Barnes,England,Attack,Centre-Forward,1989-10-30,premier-league,6225000.0,6225000,6225000
4,2010,63200,Ashley Barnes,England,Attack,Centre-Forward,1989-10-30,premier-league,90000.0,90000,90000


In [394]:
TopTier_2022_WC_Eligble_Players = TopTier_2022_WC_Eligble_Players.drop_duplicates()

In [395]:
TopTier_2022_WC_Eligble_Players = TopTier_2022_WC_Eligble_Players.sort_values(by='market_value_mean',ascending=False)

In [396]:
TopTier_2022_WC_Eligble_Players.head(20)

,World Cup,player_id,pretty_name_x,country_of_citizenship,position,sub_position,date_of_birth,name,market_value_mean,market_value_min,market_value_max
142689,2022,342229,Kylian Mbappe,France,Attack,Centre-Forward,1998-12-20,ligue-1,175000000.0,175000000,175000000
142735,2018,28003,Lionel Messi,Argentina,Attack,Right Winger,1987-06-24,ligue-1,133333333.0,133333333,133333333
142934,2018,68290,Neymar,Brazil,Attack,Left Winger,1992-02-05,ligue-1,120833333.0,120833333,120833333
28625,2022,132098,Harry Kane,England,Attack,Centre-Forward,1993-07-28,premier-league,118333333.0,118333333,118333333
142935,2022,68290,Neymar,Brazil,Attack,Left Winger,1992-02-05,ligue-1,118250000.0,118250000,118250000
142734,2014,28003,Lionel Messi,Argentina,Attack,Right Winger,1987-06-24,ligue-1,112500000.0,112500000,112500000
19940,2022,88755,Kevin De Bruyne,Belgium,Attack,Attacking Midfield,1991-06-28,premier-league,109791666.0,109791666,109791666
17817,2022,134425,Raheem Sterling,England,Attack,Left Winger,1994-12-08,premier-league,108416666.0,108416666,108416666
61678,2018,8198,Cristiano Ronaldo,Portugal,Attack,Centre-Forward,1985-02-05,serie-a,97500000.0,97500000,97500000
100420,2022,200512,Sadio Mane,Senegal,Attack,Left Winger,1992-04-10,bundesliga,97499999.0,97499999,97499999


In [397]:
TopTier_2022_WC_Eligble_Players[TopTier_2022_WC_Eligble_Players['player_id'].isin([50202])]

,World Cup,player_id,pretty_name_x,country_of_citizenship,position,sub_position,date_of_birth,name,market_value_mean,market_value_min,market_value_max
345835,2018,50202,Eden Hazard,Belgium,Attack,Left Winger,1991-01-07,laliga,82541666.0,82541666,82541666
345836,2022,50202,Eden Hazard,Belgium,Attack,Left Winger,1991-01-07,laliga,65358333.0,65358333,65358333
345834,2014,50202,Eden Hazard,Belgium,Attack,Left Winger,1991-01-07,laliga,35791666.0,35791666,35791666
345833,2010,50202,Eden Hazard,Belgium,Attack,Left Winger,1991-01-07,laliga,6700000.0,6700000,6700000


## EA Sports FIFA Ratings

In [151]:
od.download("https://www.kaggle.com/datasets/sanjeetsinghnaik/fifa-23-players-dataset")

Skipping, found downloaded files in "./fifa-23-players-dataset" (use force=True to force download)


In [27]:
fifa23data = (r'/Users/mallon/Desktop/UCD Data Analytics for Finance/fifa-23-players-dataset/Fifa 23 Players Data.csv')
fifa23= pd.read_csv(fifa23data)

In [28]:
print(fifa23.columns)

Index(['Known As', 'Full Name', 'Overall', 'Potential', 'Value(in Euro)',
       'Positions Played', 'Best Position', 'Nationality', 'Image Link', 'Age',
       'Height(in cm)', 'Weight(in kg)', 'TotalStats', 'BaseStats',
       'Club Name', 'Wage(in Euro)', 'Release Clause', 'Club Position',
       'Contract Until', 'Club Jersey Number', 'Joined On', 'On Loan',
       'Preferred Foot', 'Weak Foot Rating', 'Skill Moves',
       'International Reputation', 'National Team Name',
       'National Team Image Link', 'National Team Position',
       'National Team Jersey Number', 'Attacking Work Rate',
       'Defensive Work Rate', 'Pace Total', 'Shooting Total', 'Passing Total',
       'Dribbling Total', 'Defending Total', 'Physicality Total', 'Crossing',
       'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys',
       'Dribbling', 'Curve', 'Freekick Accuracy', 'LongPassing', 'BallControl',
       'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
       'Shot Powe

In [29]:
fifa23 = fifa23[['Full Name', 'Overall', 'Potential', 'Value(in Euro)',
       'Positions Played', 'Best Position', 'Nationality', 'Image Link', 'Age',
       'Height(in cm)', 'TotalStats', 
       'Club Name', 'Wage(in Euro)', 'Release Clause', 'Club Position', 'Club Jersey Number',
       'Skill Moves','International Reputation', 
        'National Team Name','National Team Image Link', 'National Team Position',
       'National Team Jersey Number','Finishing','Short Passing', 'Sprint Speed', 'Stamina', 'Strength', 'Penalties']]

In [30]:
fifa23.head(5)

,Full Name,Overall,Potential,Value(in Euro),Positions Played,Best Position,Nationality,Image Link,Age,Height(in cm),...,National Team Name,National Team Image Link,National Team Position,National Team Jersey Number,Finishing,Short Passing,Sprint Speed,Stamina,Strength,Penalties
0,Lionel Messi,91,91,54000000,RW,CAM,Argentina,https://cdn.sofifa.net/players/158/023/23_60.png,35,169,...,Argentina,https://cdn.sofifa.net/flags/ar.png,RW,10,90,91,76,70,68,75
1,Karim Benzema,91,91,64000000,"CF,ST",CF,France,https://cdn.sofifa.net/players/165/153/23_60.png,34,185,...,France,https://cdn.sofifa.net/flags/fr.png,ST,19,92,89,80,82,82,84
2,Robert Lewandowski,91,91,84000000,ST,ST,Poland,https://cdn.sofifa.net/players/188/545/23_60.png,33,185,...,Poland,https://cdn.sofifa.net/flags/pl.png,ST,9,94,84,75,76,87,90
3,Kevin De Bruyne,91,91,107500000,"CM,CAM",CM,Belgium,https://cdn.sofifa.net/players/192/985/23_60.png,31,181,...,Belgium,https://cdn.sofifa.net/flags/be.png,RF,7,85,93,73,88,74,83
4,Kylian Mbappé,91,95,190500000,"ST,LW",ST,France,https://cdn.sofifa.net/players/231/747/23_60.png,23,182,...,France,https://cdn.sofifa.net/flags/fr.png,ST,10,93,85,97,87,76,80


In [31]:
fifa23['Full Name']= fifa23['Full Name'].astype(str)
cols = ['Full Name']
fifa23[cols] = fifa23[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

In [32]:
fifa23transfermarkt = pd.merge(left = fifa23, right = TopTier_2022_WC_Eligble_Players, how = 'inner', left_on='Full Name', right_on='pretty_name_x' )

## Player Ratings Based on Transfer Value

In [34]:
fifa23transfermarkt['rank'] = fifa23transfermarkt['market_value_mean'].rank(ascending=False)

In [35]:
fifa23transfermarkt['rank']= fifa23transfermarkt['rank'].astype(int)

In [36]:
fifa23transfermarkt = fifa23transfermarkt.sort_values(by='rank',ascending=True)
fifa23transfermarkt.head(10)

,Full Name,Overall,Potential,Value(in Euro),Positions Played,Best Position,Nationality,Image Link,Age,Height(in cm),...,pretty_name_x,country_of_citizenship,position,sub_position,date_of_birth,name,market_value_mean,market_value_min,market_value_max,rank
18,Kylian Mbappe,91,95,190500000,"ST,LW",ST,France,https://cdn.sofifa.net/players/231/747/23_60.png,23,182,...,Kylian Mbappe,France,Attack,Centre-Forward,1998-12-20,ligue-1,175000000.0,175000000,175000000,1
0,Lionel Messi,91,91,54000000,RW,CAM,Argentina,https://cdn.sofifa.net/players/158/023/23_60.png,35,169,...,Lionel Messi,Argentina,Attack,Right Winger,1987-06-24,ligue-1,133333333.0,133333333,133333333,2
29,Harry Kane,89,89,105500000,ST,ST,England,https://cdn.sofifa.net/players/202/126/23_60.png,28,188,...,Harry Kane,England,Attack,Centre-Forward,1993-07-28,premier-league,118333333.0,118333333,118333333,3
1,Lionel Messi,91,91,54000000,RW,CAM,Argentina,https://cdn.sofifa.net/players/158/023/23_60.png,35,169,...,Lionel Messi,Argentina,Attack,Right Winger,1987-06-24,ligue-1,112500000.0,112500000,112500000,4
14,Kevin De Bruyne,91,91,107500000,"CM,CAM",CM,Belgium,https://cdn.sofifa.net/players/192/985/23_60.png,31,181,...,Kevin De Bruyne,Belgium,Attack,Attacking Midfield,1991-06-28,premier-league,109791666.0,109791666,109791666,5
77,Raheem Sterling,86,86,72500000,"LW,RW",LW,England,https://cdn.sofifa.net/players/202/652/23_60.png,27,170,...,Raheem Sterling,England,Attack,Left Winger,1994-12-08,premier-league,108416666.0,108416666,108416666,6
32,Sadio Mane,89,89,99500000,"LM,CF",LM,Senegal,https://cdn.sofifa.net/players/208/722/23_60.png,30,174,...,Sadio Mane,Senegal,Attack,Left Winger,1992-04-10,bundesliga,97499999.0,97499999,97499999,7
239,Jadon Sancho,84,88,61500000,"LW,RW,LM",CAM,England,https://cdn.sofifa.net/players/233/049/23_60.png,22,180,...,Jadon Sancho,England,Attack,Left Winger,2000-03-25,premier-league,95270833.0,95270833,95270833,8
2,Lionel Messi,91,91,54000000,RW,CAM,Argentina,https://cdn.sofifa.net/players/158/023/23_60.png,35,169,...,Lionel Messi,Argentina,Attack,Right Winger,1987-06-24,ligue-1,93583333.0,93583333,93583333,9
203,Eden Hazard,84,84,35000000,LW,LW,Belgium,https://cdn.sofifa.net/players/183/277/23_60.png,31,175,...,Eden Hazard,Belgium,Attack,Left Winger,1991-01-07,laliga,82541666.0,82541666,82541666,10


In [37]:
print(fifa23transfermarkt.columns)

Index(['Full Name', 'Overall', 'Potential', 'Value(in Euro)',
       'Positions Played', 'Best Position', 'Nationality', 'Image Link', 'Age',
       'Height(in cm)', 'TotalStats', 'Club Name', 'Wage(in Euro)',
       'Release Clause', 'Club Position', 'Club Jersey Number', 'Skill Moves',
       'International Reputation', 'National Team Name',
       'National Team Image Link', 'National Team Position',
       'National Team Jersey Number', 'Finishing', 'Short Passing',
       'Sprint Speed', 'Stamina', 'Strength', 'Penalties', 'World Cup',
       'player_id', 'pretty_name_x', 'country_of_citizenship', 'position',
       'sub_position', 'date_of_birth', 'name', 'market_value_mean',
       'market_value_min', 'market_value_max', 'rank'],
      dtype='object')


In [38]:
fifa23transfermarkt2022WC = fifa23transfermarkt[fifa23transfermarkt['World Cup']=='2022']

##### Adding GoalKeepers to the Defence Grouping

In [119]:
fifa23transfermarkt2022WC["position"] = fifa23transfermarkt2022WC["position"].replace("Goalkeeper", "Defender")

/var/folders/2d/6361wtyx7h51kz0b56dmjd_80000gn/T/ipykernel_1817/2542873994.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fifa23transfermarkt2022WC["position"] = fifa23transfermarkt2022WC["position"].replace("Goalkeeper", "Defender")


#### Splitting every country into their own DataFrame

In [121]:
unique_countries = fifa23transfermarkt2022WC['country_of_citizenship'].unique()
unique_positions = fifa23transfermarkt2022WC['position'].unique()

In [122]:
print(unique_countries)
print(unique_positions)

['France' 'England' 'Belgium' 'Senegal' 'Argentina' 'Germany' 'Uruguay'
 'Poland' 'Spain' 'Morocco' 'Canada' 'Denmark' 'Croatia' 'Ghana'
 'Netherlands' 'Mexico' 'Switzerland' 'Portugal' 'Cameroon' 'Wales'
 'Japan' 'Serbia' 'Ecuador' 'Tunisia' 'Costa Rica' 'Australia' 'Brazil']
['Attack' 'Midfield' 'Defender']


In [123]:
fifa23transfermarkt2022WC

,Full Name,Overall,Potential,Value(in Euro),Positions Played,Best Position,Nationality,Image Link,Age,Height(in cm),...,country_of_citizenship,position,sub_position,date_of_birth,name,market_value_mean,market_value_min,market_value_max,rank,column_name
18,Kylian Mbappe,91,95,190500000,"ST,LW",ST,France,https://cdn.sofifa.net/players/231/747/23_60.png,23,182,...,France,Attack,Centre-Forward,1998-12-20,ligue-1,175000000.0,175000000,175000000,1,Attack
29,Harry Kane,89,89,105500000,ST,ST,England,https://cdn.sofifa.net/players/202/126/23_60.png,28,188,...,England,Attack,Centre-Forward,1993-07-28,premier-league,118333333.0,118333333,118333333,3,Attack
14,Kevin De Bruyne,91,91,107500000,"CM,CAM",CM,Belgium,https://cdn.sofifa.net/players/192/985/23_60.png,31,181,...,Belgium,Attack,Attacking Midfield,1991-06-28,premier-league,109791666.0,109791666,109791666,5,Attack
77,Raheem Sterling,86,86,72500000,"LW,RW",LW,England,https://cdn.sofifa.net/players/202/652/23_60.png,27,170,...,England,Attack,Left Winger,1994-12-08,premier-league,108416666.0,108416666,108416666,6,Attack
32,Sadio Mane,89,89,99500000,"LM,CF",LM,Senegal,https://cdn.sofifa.net/players/208/722/23_60.png,30,174,...,Senegal,Attack,Left Winger,1992-04-10,bundesliga,97499999.0,97499999,97499999,7,Attack
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4806,Bryant Nieling,62,74,875000,CB,CB,Netherlands,https://cdn.sofifa.net/players/271/215/23_60.png,19,193,...,Netherlands,Defender,Centre-Back,2002-10-31,eredivisie,25000.0,25000,25000,5112,Defender
5094,Brett Minnema,56,67,275000,GK,GK,Netherlands,https://cdn.sofifa.net/players/262/266/23_60.png,20,192,...,Netherlands,Defender,NaN,2002-01-06,eredivisie,25000.0,25000,25000,5112,Goalkeeper
5095,Max Wolfs,56,66,275000,GK,GK,Netherlands,https://cdn.sofifa.net/players/271/313/23_60.png,20,190,...,Netherlands,Defender,NaN,2002-05-04,eredivisie,25000.0,25000,25000,5112,Goalkeeper
5006,Guus Gertsen,59,72,500000,"CB,RB",CB,Netherlands,https://cdn.sofifa.net/players/262/601/23_60.png,19,189,...,Netherlands,Defender,Centre-Back,2003-04-16,eredivisie,25000.0,25000,25000,5112,Defender


In [124]:
# creating a dicttionary of each team split into seperate dataframes
WorldCupTeams = {}


# Split the DataFrame into multiple DataFrames based on the unique values

for value1 in unique_countries:
  for value2 in unique_positions:
    value_df = fifa23transfermarkt2022WC[(fifa23transfermarkt2022WC['country_of_citizenship'] == value1) & (fifa23transfermarkt2022WC['position'] == value2)]
    WorldCupTeams[(value1, value2)] = value_df

#sets a dataframe for each variation of country and each position

for key, value in WorldCupTeams.items():
  # Create a DataFrame with the same name as the key
  globals()["_".join(key)] = pd.DataFrame(value)

In [125]:
Belgium_Defender.head(5)

,Full Name,Overall,Potential,Value(in Euro),Positions Played,Best Position,Nationality,Image Link,Age,Height(in cm),...,country_of_citizenship,position,sub_position,date_of_birth,name,market_value_mean,market_value_min,market_value_max,rank,column_name
20,Thibaut Courtois,90,91,90000000,GK,GK,Belgium,https://cdn.sofifa.net/players/192/119/23_60.png,30,199,...,Belgium,Defender,NaN,1992-05-11,laliga,60604166.0,60604166,60604166,34,Goalkeeper
941,Timothy Castagne,78,80,16000000,"RB,LB",RWB,Belgium,https://cdn.sofifa.net/players/222/501/23_60.png,26,185,...,Belgium,Defender,Right-Back,1995-12-05,premier-league,22499999.0,22499999,22499999,259,Defender
375,Toby Alderweireld,82,82,14500000,CB,CB,Belgium,https://cdn.sofifa.net/players/184/087/23_60.png,33,186,...,Belgium,Defender,Centre-Back,1989-03-02,jupiler-pro-league,20041666.0,20041666,20041666,291,Defender
962,Thomas Meunier,78,78,12000000,"RB,RM",RB,Belgium,https://cdn.sofifa.net/players/202/371/23_60.png,30,191,...,Belgium,Defender,Right-Back,1991-09-12,bundesliga,17562500.0,17562500,17562500,342,Defender
191,Koen Casteels,84,85,31500000,GK,GK,Belgium,https://cdn.sofifa.net/players/192/984/23_60.png,30,197,...,Belgium,Defender,NaN,1992-06-25,bundesliga,11500000.0,11500000,11500000,593,Goalkeeper


In [92]:
def play_game(team1, team2, team1_rating, team2_rating):
  # Determine the probability that team1 wins based on the ratings
  p1 = 1.0 / (1.0 + 10.0**((team2_rating - team1_rating) / 400.0))

  # Use the probability to determine the outcome of the game
  if random.random() <= p1:
    # Team 1 wins
    return team1, p1 *100
  else:
    # Team 2 wins
    return team2

In [93]:
print(play_game("Belgium", "Brazil", Belgium["market_value_mean"].mean(), Brazil["market_value_mean"].mean()))
print(play_game("Belgium", "Brazil", Belgium["Age"].mean(), Brazil["Age"].mean()))

('Belgium', 100.0)
('Belgium', 49.69680689402655)


In [94]:
print(Belgium["market_value_mean"].mean())
print(Brazil["market_value_mean"].mean())

4445412.232931727
2162499.6


In [95]:
print(Belgium["Age"].mean())
print(Brazil["Age"].mean())

24.29317269076305
26.4


In [ ]:
def play_game(team1, team2, team1_def, team1_mid, team1_att, team2_def, team2_mid, team2_att):
  # Calculate the overall rating for each team's attack and midfield
  team1_att_mid_rating = (team1_att + team1_mid) / 2
  team2_att_mid_rating = (team2_att + team2_mid) / 2

  # Calculate the probability that team1's attack and midfield win against team2's defense
  p1 = 1.0 / (1.0 + 10.0**((team2_def - team1_att_mid_rating) / 400.0))

  # Use the probability to determine the outcome of the game
  if random.random() <= p1:
    # Team 1's attack and midfield win
    # Now we need to determine the outcome of the game based on the ratings of the two teams' midfields
    p1 = 1.0 / (1.0 + 10.0**((team2_mid - team1_mid) / 400.0))
    if random.random() <= p1:
      # Team 1 wins
      return team1
    else:
      # Team 2 wins
      return team2
  else:
    # Team 2's defense wins
    # Now we need to determine the outcome of the game based on the ratings of the two teams' midfields
    p1 = 1.0 / (1.0 + 10.0**((team2_mid - team1_mid) / 400.0))
    if random.random() <= p1:
      # Team 2 wins
      return team2
    else:
      # Team 1 wins
      return team1

In [160]:
England_Attack.head(15)

,Full Name,Overall,Potential,Value(in Euro),Positions Played,Best Position,Nationality,Image Link,Age,Height(in cm),...,country_of_citizenship,position,sub_position,date_of_birth,name,market_value_mean,market_value_min,market_value_max,rank,column_name
29,Harry Kane,89,89,105500000,ST,ST,England,https://cdn.sofifa.net/players/202/126/23_60.png,28,188,...,England,Attack,Centre-Forward,1993-07-28,premier-league,118333333.0,118333333,118333333,3,Attack
77,Raheem Sterling,86,86,72500000,"LW,RW",LW,England,https://cdn.sofifa.net/players/202/652/23_60.png,27,170,...,England,Attack,Left Winger,1994-12-08,premier-league,108416666.0,108416666,108416666,6,Attack
239,Jadon Sancho,84,88,61500000,"LW,RW,LM",CAM,England,https://cdn.sofifa.net/players/233/049/23_60.png,22,180,...,England,Attack,Left Winger,2000-03-25,premier-league,95270833.0,95270833,95270833,8,Attack
436,Marcus Rashford,81,85,37000000,"LW,ST",ST,England,https://cdn.sofifa.net/players/231/677/23_60.png,24,186,...,England,Attack,Left Winger,1997-10-31,premier-league,75333333.0,75333333,75333333,17,Attack
164,Jack Grealish,84,84,46000000,"LW,LM,CAM",LW,England,https://cdn.sofifa.net/players/206/517/23_60.png,26,180,...,England,Attack,Left Winger,1995-09-10,premier-league,54237500.0,54237500,54237500,47,Attack
237,Mason Mount,84,87,57500000,"CAM,RW",CAM,England,https://cdn.sofifa.net/players/233/064/23_60.png,23,180,...,England,Attack,Attacking Midfield,1999-01-10,premier-league,53312500.0,53312500,53312500,52,Attack
292,James Maddison,82,84,38000000,"CAM,RM",CAM,England,https://cdn.sofifa.net/players/220/697/23_60.png,25,175,...,England,Attack,Attacking Midfield,1996-11-23,premier-league,51812499.0,51812499,51812499,53,Attack
1082,Dele Alli,77,80,14000000,"CAM,CM",ST,England,https://cdn.sofifa.net/players/211/117/23_60.png,26,188,...,England,Attack,Attacking Midfield,1996-04-11,super-lig,48708333.0,48708333,48708333,58,Attack
330,Tammy Abraham,82,85,42000000,ST,ST,England,https://cdn.sofifa.net/players/231/352/23_60.png,24,195,...,England,Attack,Centre-Forward,1997-10-02,serie-a,38645833.0,38645833,38645833,98,Attack
624,Jarrod Bowen,80,83,28000000,"RM,ST",LM,England,https://cdn.sofifa.net/players/224/371/23_60.png,25,175,...,England,Attack,Right Winger,1996-12-20,premier-league,26229166.0,26229166,26229166,195,Attack


In [262]:
def play_game(team1_def_rating,team1_mid_rating,team1_att_rating,team2_def_rating,team2_mid_rating,team2_att_rating):


  team1_rounds = 0
  team2_rounds = 0

  # Check if team 1's attack is greater than team 2's defense
  if (team1_att_rating["Overall"].mean() > team2_def_rating["Overall"].mean()):
    # Calculate the difference in ratings
    rating_diff = team1_att_rating["Overall"].mean() - team2_def_rating["Overall"].mean()
    # Calculate the probability of a normal distribution with a mean equal to the difference in ratings
    # and a standard deviation of 1
    probability = norm.cdf(rating_diff, 1)
    # Generate a random number between 0 and 1
    rand = random.random()
    # If the random number is less than the probability of winning, team 1 wins a round
    if rand < probability:
      team1_rounds += 1

    
  # Check if team 2's attack is greater than team 1's defense
  if (team2_att_rating["Overall"].mean() > team1_def_rating["Overall"].mean()):
    # Calculate the difference in ratings
    rating_diff = team2_att_rating["Overall"].mean() - team1_def_rating["Overall"].mean()
    # Calculate the probability of a normal distribution with a mean equal to the difference in ratings
    # and a standard deviation of 1
    probability = norm.cdf(rating_diff, 1)
    # Generate a random number between 0 and 1
    rand = random.random()
    # If the random number is less than the probability of winning, team 1 wins a round
    if rand < probability:
      team2_rounds += 1
    
 

   # Check if team 2's midfield is greater than team 1's midfield
  if (team2_mid_rating["Overall"].mean() > team1_mid_rating["Overall"].mean()):
    # Calculate the difference in ratings
    rating_diff = team2_mid_rating["Overall"].mean() - team1_mid_rating["Overall"].mean()
    # Calculate the probability of a normal distribution with a mean equal to the difference in ratings
    # and a standard deviation of 1
    probability = norm.cdf(rating_diff, 1)
    # Generate a random number between 0 and 1
    rand = random.random()
    # If the random number is less than the probability of winning, team 2 wins a round
    if rand < probability:
      team2_rounds += 1
    
    
    
    print(team1_rounds,team2_rounds)
    print(round(team1_att_rating["Overall"].mean(),2) , "Team1 Attack") 
    print(round(team1_mid_rating["Overall"].mean(),2) , "Team1 Midfield")
    print(round(team1_def_rating["Overall"].mean(),2) , "Team1 Defending")
    print(round(team2_def_rating["Overall"].mean(),2) , "Team2 Defending")
    print(round(team2_mid_rating["Overall"].mean(),2) , "Team2 Midfield") 
    print(round(team2_att_rating["Overall"].mean(),2) , "Team2 Attacking") 
    

  if team1_rounds > team2_rounds:
    return "Team 1 wins!"
  elif team1_rounds < team2_rounds:
    return "Team 2 wins!"
  else:
    return "It's a draw!"


In [263]:
play_game(Belgium_Defender,Belgium_Midfield,Belgium_Attack,England_Defender,England_Midfield,England_Attack)

0 2
69.68 Team1 Attack
68.14 Team1 Midfield
67.18 Team1 Defending
71.18 Team2 Defending
73.28 Team2 Midfield
72.69 Team2 Attacking


'Team 2 wins!'

In [264]:
play_game(Germany_Defender,Germany_Midfield,Germany_Attack,England_Defender,England_Midfield,England_Attack)

0 1
71.61 Team1 Attack
72.53 Team1 Midfield
70.96 Team1 Defending
71.18 Team2 Defending
73.28 Team2 Midfield
72.69 Team2 Attacking


'Team 2 wins!'

In [265]:
play_game(Germany_Defender,Germany_Midfield,Germany_Attack,Wales_Defender,Wales_Midfield,Wales_Attack)

"It's a draw!"

In [266]:
Wales_Attack

,Full Name,Overall,Potential,Value(in Euro),Positions Played,Best Position,Nationality,Image Link,Age,Height(in cm),...,country_of_citizenship,position,sub_position,date_of_birth,name,market_value_mean,market_value_min,market_value_max,rank,column_name
1385,David Brooks,76,79,10000000,"RW,RM",CAM,Wales,https://cdn.sofifa.net/players/220/196/23_60.png,24,180,...,Wales,Attack,Right Winger,1997-07-08,premier-league,19749999.0,19749999,19749999,296,Attack
1074,Daniel James,77,82,16000000,"RM,ST,LM",RM,Wales,https://cdn.sofifa.net/players/232/104/23_60.png,24,171,...,Wales,Attack,Right Winger,1997-11-10,premier-league,17791666.0,17791666,17791666,336,Attack
1369,Harry Wilson,76,79,10000000,"RM,RW,CAM",CAM,Wales,https://cdn.sofifa.net/players/220/710/23_60.png,25,173,...,Wales,Attack,Right Winger,1997-03-22,premier-league,16531250.0,16531250,16531250,373,Attack
1631,Brennan Johnson,75,85,12500000,"RW,ST",RM,Wales,https://cdn.sofifa.net/players/251/421/23_60.png,21,181,...,Wales,Attack,Second Striker,2001-05-23,premier-league,9516666.0,9516666,9516666,712,Attack
2403,Rabbi Matondo,73,82,7000000,"LW,RW,ST",RM,Wales,https://cdn.sofifa.net/players/243/392/23_60.png,21,175,...,Wales,Attack,Right Winger,2000-09-09,scottish-premiership,6125000.0,6125000,6125000,1035,Attack
2491,Tom Lawrence,73,73,3100000,"LW,ST,CAM",LW,Wales,https://cdn.sofifa.net/players/211/513/23_60.png,28,180,...,Wales,Attack,Left Winger,1994-01-13,scottish-premiership,4883333.0,4883333,4883333,1226,Attack
2481,Kieffer Moore,73,73,3000000,ST,ST,Wales,https://cdn.sofifa.net/players/216/627/23_60.png,29,196,...,Wales,Attack,Centre-Forward,1992-08-08,premier-league,3385416.0,3385416,3385416,1494,Attack
4505,Christian Doidge,65,65,725000,ST,ST,Wales,https://cdn.sofifa.net/players/224/965/23_60.png,29,185,...,Wales,Attack,Centre-Forward,1992-08-25,scottish-premiership,777083.0,777083,777083,2882,Attack
4389,Marley Watkins,65,65,600000,ST,CF,Wales,https://cdn.sofifa.net/players/189/776/23_60.png,31,178,...,Wales,Attack,Right Winger,1990-10-17,scottish-premiership,546875.0,546875,546875,3234,Attack
4749,Alex Samuel,63,63,525000,ST,ST,Wales,https://cdn.sofifa.net/players/222/848/23_60.png,26,183,...,Wales,Attack,Centre-Forward,1995-09-20,scottish-premiership,183333.0,183333,183333,4342,Attack


In [282]:
bale = fifa23transfermarkt2022WC[fifa23transfermarkt2022WC["Full Name"]== "Alex Samuel"]

In [283]:
bale

,Full Name,Overall,Potential,Value(in Euro),Positions Played,Best Position,Nationality,Image Link,Age,Height(in cm),...,country_of_citizenship,position,sub_position,date_of_birth,name,market_value_mean,market_value_min,market_value_max,rank,column_name
4749,Alex Samuel,63,63,525000,ST,ST,Wales,https://cdn.sofifa.net/players/222/848/23_60.png,26,183,...,Wales,Attack,Centre-Forward,1995-09-20,scottish-premiership,183333.0,183333,183333,4342,Attack
